In [1]:
%display latex
from pylie import *
from collections.abc import Iterable
import functools
from operator import mul, sub
from pprint import pprint
from IPython.core.debugger import set_trace


In [2]:
def reduce(e1: Differential_Polynomial,e2: Differential_Polynomial, context:Context)->Differential_Polynomial:
    assert e2.is_monic()
    def _order (der):
        if der != 1:
            return pylie.order_of_derivative(der)
        else :
            return [0]*len(context._independent)
    def func(e):    
        try:
            return e.operator().function()
        except AttributeError:
            return e.operator()      
    print ("--------------------------------------- entering reduce")    
    #set_trace()
    ld = e2.Lder()
    e2_order = _order (ld)    
    while True:
        for t in e1._p:
            reduction_found = False
            d = t._d
            c = t._coeff
            #t.show()
            #ld.show()
            if func(ld) != func(d):
                break
            e1_order = _order (d)
            dif = []
            for a, b in zip (e1_order, e2_order):
                dif += [a - b]
            if all (map (lambda h: h == 0, dif)) :                
                e1 = e1 - e2 * c                       
                return e1
            if all (map (lambda h: h >= 0, dif)):         
                variables_to_diff = []
                for i in range (len(context._independent)):
                    if dif [i] != 0:
                        variables_to_diff.extend ([context._independent[i]]*abs(dif[i]))      
                #set_trace ()
                s  = sum([c * _._coeff * diff(_._d,  *variables_to_diff) for _ in e2._p])
                s  = Differential_Polynomial (s, context)
                s.show()
                #set_trace()
                e1 = e1 - s
                return e1
            print ("no reduction")
        if not reduction_found:
            return e1        
    return e1

In [3]:
#indeterminates = (x, y) = var("x y")
#R=PolynomialRing(QQ, indeterminates)
#z  = function ("z")(*indeterminates)
#ctx = Context((z,), indeterminates)
#print("*"*70)
#e1 = diff(z,y)-x**2/y**2*diff(z,x)-(x-y)*z(x,y)/y**2
#e1_dp = Differential_Polynomial (e1, ctx)
#print (e1_dp)
#print("*"*70)
#f1 = diff(z, x) + z(x,y)/x
#f1_dp = Differential_Polynomial (f1, ctx)
#f1_dp.is_monic()
#print (f1_dp)
#print("*"*70)
#f2 = diff(z, y) + z(x,y)/y
#f2_dp = Differential_Polynomial (f2, ctx)
#print (f2_dp)
#r1 = reduce (e1_dp, f1_dp, ctx)
#r1
#r1._orig
#r2 = reduce (r1, f2_dp, ctx)

In [4]:
def Lfunc(e):
    return e

In [5]:
def Lcml(e):
    '''Least common left multiply, p.23'''
    return 0

In [6]:
def Lcm(e):
    return 0

In [7]:
def CompleteSystem(S, context):
    return S

In [8]:
def FindIntegrableConditions(S, context):
    return S

In [9]:
def ReduceIntegrabilityConditions(S, context):
    return S

In [10]:

def Reorder (S, context, ascending = False):
    res=sorted(S)
    if ascending :
        res.reverse()
    return res

In [11]:
var ("x y")
z = function("z")(x,y)
w = function("w")(x,y)
ctx = Context((w,z), (x,y), MatrixOrder.Mgrlex)
f1 =diff(w, y) + x*diff(z,y)/(2*y*(x**2+y)) - w(x,y)/y
f1.show()

-w(x, y)/y + 1/2*x*diff(z(x, y), y)/((x^2 + y)*y) + diff(w(x, y), y)

In [12]:
f2=diff(z,x,y) + y*diff(w,y)/x + 2*y*diff(z, x)/x
f2.show()

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

In [13]:
f3 = diff(w, x,y) - 2*x*diff(z, x,2)/y - x*diff(w,x)/y**2
f3.show()

-2*x*diff(z(x, y), x, x)/y - x*diff(w(x, y), x)/y^2 + diff(w(x, y), x, y)

In [14]:
f4 = diff(w, x,y) + diff(z, x,y) + diff(w, y)/(2*y) - diff(w,x)/y + x* diff(z, y)/y - w(x,y)/(2*y**2)
f4.show()

x*diff(z(x, y), y)/y - diff(w(x, y), x)/y + 1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), x, y) + diff(z(x, y), x, y)

In [15]:
f5 =diff(w,y,y) + diff(z,x,y) - diff(w, y)/y + w(x,y)/(y**2)
f5.show()

-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

In [16]:
def Autoreduce(S, context):  
    i = 0
    while True:
        p = S[i]
        r = S[i+1:]
        if not r:
            S = Reorder(S, context,True)
            return S[:]    
        print ("=====================================================", len(r))
        if len(r) == 3:
            set_trace()
        have_reduced = False
        #set_trace()
        checkidx = i+1
        for _r in r:
            #print ("Try to reduce {} with {}".format(S[checkidx], S[i].Lder()))
            res = reduce (_r, p, context)
            if res != _r:
                have_reduced = True                
            S[checkidx] = res
            checkidx += 1
        #print("This is the new System before reorder")   
        #for _ in S:          
        #    _.show()
        S = Reorder(S, context, ascending = True)
        print("This is the new System........")
        for _ in S:          
            _.show()
        if not have_reduced:
            # no reduction done
            i += 1        
        else:
            i = 0
            

In [17]:
def JanetBasis(S, context):
    """List of homogenous PDE's + order context"""
    if not isinstance(S, Iterable):
        # bad criterion
        S = [S]
    S = [Differential_Polynomial(s, context) for s in S]
    for _ in S:
        _.show()
    print ("....................................")
    S = Reorder(S, context, ascending = True)
    for _ in S:
        _.show()
    while 1:
        S = Autoreduce (S, context)
        #S = CompleteSystem (S, context)
        #f = FindIntegrableConditions(S, context)
        #c = ReduceIntegrabilityConditions(S, context)
        #if all ([_ == 0 for _ in c]) or not c:
        #    return S
        #S += c
        #S = Reorder (S, context)
        return S[:]

In [ ]:
checkS=JanetBasis([f3,f1,f5, f2,f4], ctx)

-2*x*diff(z(x, y), x, x)/y - x*diff(w(x, y), x)/y^2 + diff(w(x, y), x, y)

1/2*x*diff(z(x, y), y)/(x^2*y + y^2) - w(x, y)/y + diff(w(x, y), y)

-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

x*diff(z(x, y), y)/y - diff(w(x, y), x)/y + 1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), x, y) + diff(z(x, y), x, y)

...

(x^3 + x*y)*w(x, y)/(2*x^2*y^3 - x^3*y + 2*y^4 - 2*x*y^2) + 2*(2*x^2*y^2 + 2*y^3 - x*y)*diff(z(x, y), x)/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) - 2*(x^4 + x^2*y)*diff(z(x, y), y)/(2*x^2*y^2 - x^3 + 2*y^3 - 2*x*y) + diff(w(x, y), y)

1/4*(16*(2*x^2 + x)*y^8 - x^8 + 4*(32*x^4 + 12*x^3 - 2*x^2 - x)*y^7 + 4*(48*x^6 + 12*x^5 - 10*x^4 - 10*x^3 - 7*x^2)*y^6 + 2*(64*x^8 + 8*x^7 - 44*x^6 - 50*x^5 - 32*x^4 + 2*x^3 + 3*x^2)*y^5 + 4*(8*x^10 - 26*x^8 - 24*x^7 - 11*x^6 + 2*x^5 + 6*x^4 + 4*x^3)*y^4 - 2*(32*x^10 + 16*x^9 + 4*x^8 - 2*x^7 - 21*x^6 - 12*x^5 + x^3)*y^3 - (16*x^12 - 32*x^8 - 8*x^7 + x^5 + 4*x^4)*y^2 + 4*(2*x^10 - x^6)*y)*w(x, y)/(12*x^2*y^9 + 4*y^10 + x^4*y^5 + 4*(3*x^4 - x)*y^8 + 4*(x^6 - 2*x^3)*y^7 - (4*x^5 - x^2)*y^6) - 1/4*(16*(2*x - 1)*y^9 + 4*(32*x^3 - 12*x^2 - 2*x - 1)*y^8 + 4*(48*x^5 - 12*x^4 - 10*x^3 - 10*x^2 + 5*x)*y^7 + 2*(64*x^7 - 8*x^6 - 44*x^5 - 50*x^4 + 24*x^3 + 2*x^2 + x)*y^6 - x^6 + 4*(8*x^9 - 26*x^7 - 24*x^6 + 9*x^5 + 2*x^4 + 2*x^3 - 3*x^2)*y^5 - 2*(32*x^9 + 16*x^8 - 4*x^7 - 2*x^6 - 7*x^5 + 12*x^4 - x^2)*y^4 - (16*x^11 - 8*x^7 + 12*x^6 - 7*x^4 - 6*x^3)*y^3 + (8*x^6 + 8*x^5 - x^3)*y^2 + (4*x^8 + x^7 - 2*x^4)*y)*diff(w(x, y), y)/(12*x^2*y^7 + 4*y^8 + x^4*y^3 + 4*(3*x^4 - x)*y^6 + 4*(x^6 - 2*x^3)*y^5 - (4*x^5 - x^2)*y^4) - 1/2*(16*(2*x - 1)*y^8 + 4*(32*x^3 - 12*x^2 - 2*x - 1)*y^7 - x^7 + 4*(48*x^5 - 12*x^4 - 10*x^3 - 10*x^2 + x)*y^6 + 2*(64*x^7 - 8*x^6 - 44*x^5 - 50*x^4 + 2*x^2 + 3*x)*y^5 + 4*(8*x^9 - 26*x^7 - 24*x^6 - 3*x^5 + 2*x^4 + 6*x^3 + 2*x^2)*y^4 - 2*(32*x^9 + 16*x^8 + 4*x^7 - 2*x^6 - 21*x^5 - 8*x^4 + x^2)*y^3 - (16*x^11 - 32*x^7 - 8*x^6 + x^4 + 4*x^3)*y^2 + 4*(2*x^9 - x^5)*y)*diff(z(x, y), x)/(12*x^2*y^6 + 4*y^7 + x^4*y^2 + 4*(3*x^4 - x)*y^5 + 4*(x^6 - 2*x^3)*y^4 - (4*x^5 - x^2)*y^3) + diff(w(x, y), x)

-1/2*(x^4 - 2*(3*x^2 + x)*y^3 - 2*y^4 - (8*x^4 + 2*x^3 - x)*y^2 - 2*(2*x^6 - x^2)*y)*diff(w(x, y), y)/(4*x^2*y^3 - x^3*y + 2*y^4 + (2*x^4 - x)*y^2) - (x^4 - 2*(3*x^2 + x)*y^3 - 2*y^4 - (8*x^4 + 2*x^3 - x)*y^2 - 2*(2*x^6 - x^2)*y)*diff(z(x, y), x)/(4*x^2*y^3 - x^3*y + 2*y^4 + (2*x^4 - x)*y^2) + diff(w(x, y), x)

-8*(16*x^2*y^9 + x^6*y^4 + 4*y^10 + 4*(6*x^4 - x)*y^8 + 4*(4*x^6 - 3*x^3)*y^7 + (4*x^8 - 12*x^5 + x^2)*y^6 - 2*(2*x^7 - x^4)*y^5)*diff(w(x, y), x)/(x^9 - 16*(2*x^3 + x^2)*y^8 - 4*(32*x^5 + 12*x^4 - 2*x^3 - x^2)*y^7 - 4*(48*x^7 + 12*x^6 - 10*x^5 - 10*x^4 - 7*x^3)*y^6 - 2*(64*x^9 + 8*x^8 - 44*x^7 - 50*x^6 - 32*x^5 + 2*x^4 + 3*x^3)*y^5 - 4*(8*x^11 - 26*x^9 - 24*x^8 - 11*x^7 + 2*x^6 + 6*x^5 + 4*x^4)*y^4 + 2*(32*x^11 + 16*x^10 + 4*x^9 - 2*x^8 - 21*x^7 - 12*x^6 + x^4)*y^3 + (16*x^13 - 32*x^9 - 8*x^8 + x^6 + 4*x^5)*y^2 - 4*(2*x^11 - x^7)*y) - 2*(128*x^2*y^10 + 32*y^11 + 48*(4*x^4 - x)*y^9 + x^8*y + 4*(32*x^6 - 40*x^3 + x)*y^8 + 4*(8*x^8 - 48*x^5 + 5*x^3 + 7*x^2)*y^7 - 4*(24*x^7 - 11*x^5 - 19*x^4 + x^2)*y^6 - 2*(8*x^9 - 26*x^7 - 34*x^6 + 6*x^4 + 5*x^3)*y^5 + (32*x^9 + 20*x^8 - 16*x^6 - 22*x^5 + x^3)*y^4 + (8*x^11 - 12*x^8 - 14*x^7 + x^5 + 2*x^4)*y^3 - (4*x^10 + 2*x^9 - 3*x^6)*y^2)*diff(w(x, y), y)/(x^9 - 16*(2*x^3 + x^2)*y^8 - 4*(32*x^5 + 12*x^4 - 2*x^3 - x^2)*y^7 - 4*(48*x^7 + 12*x^6 - 10*x^5 - 10*x^4 - 7*x^3)*y^6 - 2*(64*x^9 + 8*x^8 - 44*x^7 - 50*x^6 - 32*x^5 + 2*x^4 + 3*x^3)*y^5 - 4*(8*x^11 - 26*x^9 - 24*x^8 - 11*x^7 + 2*x^6 + 6*x^5 + 4*x^4)*y^4 + 2*(32*x^11 + 16*x^10 + 4*x^9 - 2*x^8 - 21*x^7 - 12*x^6 + x^4)*y^3 + (16*x^13 - 32*x^9 - 8*x^8 + x^6 + 4*x^5)*y^2 - 4*(2*x^11 - x^7)*y) - 32*(16*x^2*y^10 + x^6*y^5 + 4*y^11 + 4*(6*x^4 - x)*y^9 + 4*(4*x^6 - 3*x^3)*y^8 + (4*x^8 - 12*x^5 + x^2)*y^7 - 2*(2*x^7 - x^4)*y^6)*diff(z(x, y), x)/(x^9 - 16*(2*x^3 + x^2)*y^8 - 4*(32*x^5 + 12*x^4 - 2*x^3 - x^2)*y^7 - 4*(48*x^7 + 12*x^6 - 10*x^5 - 10*x^4 - 7*x^3)*y^6 - 2*(64*x^9 + 8*x^8 - 44*x^7 - 50*x^6 - 32*x^5 + 2*x^4 + 3*x^3)*y^5 - 4*(8*x^11 - 26*x^9 - 24*x^8 - 11*x^7 + 2*x^6 + 6*x^5 + 4*x^4)*y^4 + 2*(32*x^11 + 16*x^10 + 4*x^9 - 2*x^8 - 21*x^7 - 12*x^6 + x^4)*y^3 + (16*x^13 - 32*x^9 - 8*x^8 + x^6 + 4*x^5)*y^2 - 4*(2*x^11 - x^7)*y) + diff(z(x, y), y, y)

===================================================== 3
> <ipython-input-16-f640c8c307c8>(12)Autoreduce()
     10         if len(r) == Integer(3):
     11             set_trace()
---> 12         have_reduced = False
     13         #set_trace()
     14         checkidx = i+Integer(1)



In [ ]:
for _ in checkS:
    _._orig.show()

In [ ]:
g1 = Differential_Polynomial (diff(z,y,y) + diff(z,y)/(2*y), ctx)
g1._orig

In [ ]:
g2 = Differential_Polynomial(diff(w,x,x) + 4*y*y*diff(w,y) - 8*y*y*diff(z, x) - 8*y*w(x,y), ctx)
g2._orig

In [ ]:
g3 = Differential_Polynomial(diff(w,x,y) - diff(z,x,x)/2 - diff(w,x)/(2*y) - 6*y**2*diff(z,y), ctx)
g3._orig

In [ ]:
g4 = Differential_Polynomial(diff(w,y,y) - 2*diff(z,x,y)- diff(w,y)/(2*y) + w(x,y)/(2*y**2), ctx)
g4._orig

In [ ]:
f1._orig

In [ ]:
f4._orig

In [ ]:
print(f4.Lder(), f1.Lder())

In [ ]:
f4a = reduce(f4, f1, ctx)

In [ ]:
for _ in f4a._p:
   print (_)

In [ ]:
f4a._orig

In [ ]:
wuwu=f4a._p[-1]._coeff.operands()[0]*f4a._p[-1]._coeff.operands()[1].expand().simplify_full()

In [ ]:
f4b = reduce(f4a, f1, ctx)

In [ ]:
f4b._orig

In [ ]:
functools.reduce(operator.mul, f4b._p[1]._coeff.operands()).expand().simplify_full()

In [ ]:
f4c = reduce(f4b, f2, ctx)

In [ ]:
f4c._p[2]._coeff.expand().simplify_full()

In [ ]:
f4d = reduce (f4c, f1, ctx)

In [ ]:
f4d._p[1]._coeff.expand().simplify_full()

In [ ]:
f4d._p[0]._d

In [ ]:
wuwu.collect(y)
# correct according Example 2.35

In [ ]:
wuwu=f4a._p[-2]._coeff

In [ ]:
wuwu

In [ ]:
wuwu=f4a._p[-2]._coeff.operands()

In [ ]:
wuwu

In [ ]:
wawa=wuwu[0]*wuwu[1]

In [ ]:
wawa.expand().simplify_full()

In [ ]:
# wawa is a

In [ ]:
f1._orig.show()

In [ ]:
f2._orig.show()

In [ ]:
f3._orig.show()

In [ ]:
f4._orig.show()

In [ ]:
 f5._orig.show()

In [ ]:
f5a=(f5._orig - (-1/y)*f1._orig).expand().simplify(); f5a.show()

In [ ]:
f2a=(f2._orig - (y/x)* f1._orig).expand().collect(diff(z,y)); f2a.show()

In [ ]:
(f4._orig - diff(f1._orig, x)).expand().collect(diff(z, y)).show()